#Book_Atrasos

##configurações iniciais

In [1]:
!pip install spark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.7/351.7 kB 30.6 MB/s eta 0:00:00


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import date_format, col, count, round, cast, desc,to_timestamp, concat_ws, substring, lit, lpad, countDistinct, row_number, isnan, when, corr, first, max as Fmax, min as Fmin, sum as Fsum, to_timestamp, to_date, concat
from pyspark.sql import types as T
from pyspark.sql import functions as F
from pyspark.sql.window import Window

import builtins

In [3]:
spark = (
    SparkSession.builder
    .appName("book_atraso")

    # =========================
    # PARALELISMO
    # =========================
    .config("spark.sql.shuffle.partitions", "120")
    .config("spark.default.parallelism", "16")

    # =========================
    # MEMÓRIA (RAM 50GB)
    # =========================
    .config("spark.driver.memory", "32g")
    .config("spark.executor.memory", "32g")
    .config("spark.memory.fraction", "0.85")
    .config("spark.memory.storageFraction", "0.25")

    # =========================
    # EXECUÇÃO
    # =========================
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
    .config("spark.sql.adaptive.skewJoin.enabled", "true")

    # =========================
    # JOINS
    # =========================
    .config("spark.sql.autoBroadcastJoinThreshold", "100m")

    # =========================
    # SERIALIZAÇÃO
    # =========================
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config("spark.kryoserializer.buffer.max", "512m")

    # =========================
    # CACHE / COLUMNAR
    # =========================
    .config("spark.sql.inMemoryColumnarStorage.compressed", "true")
    .config("spark.sql.inMemoryColumnarStorage.batchSize", "20000")

    # =========================
    # GC
    # =========================
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC")
    .config("spark.driver.extraJavaOptions", "-XX:+UseG1GC")

    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")

In [4]:
from pyspark.sql import functions as F

In [5]:
from pyspark.sql import types as T

In [6]:
from pyspark.sql.window import Window

##Importação da base de público válida

In [8]:
# Instalação e configuração de variaveis ambiente para utilizar Google Drive
# Se utilizar o Colab ajustar para True para instalação dos pre-requisitos
colab = True

if colab==True:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


antes de iniciar o book_atrasos, traremos a base score full com todos CPFs que iremos considerar, para cruzar com os pagamentos e reduzir registros que não interessam para o público alvo (base atrasos tem 21,8 MM de registros e cai para 15,8 MM após filtro).

In [9]:
df_base_score_full = spark.read.parquet("/content/drive/MyDrive/hackathon/projeto_hackathon/01_INGESTAO/base_score/stg/base_score_full/SAFRA=202410",
                                        "/content/drive/MyDrive/hackathon/projeto_hackathon/01_INGESTAO/base_score/stg/base_score_full/SAFRA=202411",
                                        "/content/drive/MyDrive/hackathon/projeto_hackathon/01_INGESTAO/base_score/stg/base_score_full/SAFRA=202412",
                                        "/content/drive/MyDrive/hackathon/projeto_hackathon/01_INGESTAO/base_score/stg/base_score_full/SAFRA=202501")

###Exploração | tratamentos | **filtros**

In [10]:
#renomear safra para safra_buerau
df_base_score_full = df_base_score_full.withColumnRenamed("SAFRA", "SAFRA_BUREAU")

In [11]:
# df_base_score_full.groupBy("NUM_CPF").count().filter("count > 1").count()

In [12]:
# print(f"Linhas totais: {df_base_score_full.count():,}")
# print(f"CPFs únicos: {df_base_score_full.select('NUM_CPF').distinct().count():,}")

In [13]:
#transformação formato variáveis numéricas
df_base_score_full = (
df_base_score_full    .withColumn("NUM_CPF", col("NUM_CPF").cast("string"))
    .withColumn("SCORE_01", col("SCORE_01").cast("double"))
    .withColumn("SCORE_02", col("SCORE_02").cast("double"))
    .withColumn("FPD", col("FPD").cast("int"))
)

In [14]:
df_cpfs_validos = df_base_score_full.select("NUM_CPF").distinct()

In [15]:
#quantidade de CPFs distintos no bureau
df_cpfs_validos.select("NUM_CPF").distinct().count()

2462481

###IMPORTAÇÃO BASE ATRASOS

In [16]:
path_rec = "/content/drive/MyDrive/hackathon/book_atraso/bases_atrasos"
df_atrasos = spark.read.parquet(path_rec)

In [17]:
# print("✅ Linhas:", df_atrasos.count())
# print("✅ Colunas:", len(df_atrasos.columns))
# df_atrasos.printSchema()
# df_atrasos.show(5)

In [18]:
# df_atrasos.select("NUM_CPF").distinct().count()

###JOIN BASES

In [19]:
#join para usar somente os CPFs válidos
df_atrasos_alvo = df_atrasos.join(df_cpfs_validos, "NUM_CPF", "inner")

In [20]:
# print("✅ Linhas:", df_atrasos_alvo.count())
# print("✅ Colunas:", len(df_atrasos_alvo.columns))
# df_atrasos_alvo.printSchema()
# df_atrasos_alvo.show(5)

In [21]:
#quantidade de CPFs correspondidos entre bureau e pagamentos
# df_atrasos_alvo.select("NUM_CPF").distinct().count()

In [22]:
#marcação da terminação nas colunas da base da base origem(_ATRASO)
cols = [F.col("NUM_CPF")] + [
    F.col(c).alias(f"{c}_ATRASO") for c in df_atrasos_alvo.columns if c != "NUM_CPF"
]
df_atrasos_alvo = df_atrasos_alvo.select(cols)

In [23]:
df_00=df_atrasos_alvo

In [24]:
def padronizar_df_atraso(df):
    """
    Padroniza colunas do book de atraso:
    - Mantém NUM_CPF sem sufixo
    - Aplica '_ATRASO' às demais
    - Converte valores e datas com tolerância a erros
    """
    # Mantém CPF e renomeia demais
    '''cols = [F.col("NUM_CPF")] + [
        F.col(c).alias(f"{c}_ATRASO") for c in df.columns if c != "NUM_CPF"
    ]
    df = df.select(cols)'''

    # Colunas numéricas: usar try_cast seguro
    num_cols = [c for c in df.columns if (("VAL_" in c) or ("NUM_" in c)) and c != "NUM_CPF"]

    for c in num_cols:
        df = df.withColumn(c, F.regexp_replace(F.col(c), ",", "."))
        df = df.withColumn(
            c,
            F.when(
                F.col(c).rlike("^[0-9.]+$"),  # mantém apenas se é número
                F.col(c).cast("double")
            ).otherwise(None)
        )

    # Colunas de data: tolerar formatos diferentes
    date_cols = [c for c in df.columns if "DAT_" in c]
    for c in date_cols:
        df = df.withColumn(
            c,
            F.coalesce(
                F.try_to_timestamp(F.col(c), F.lit("ddMMMyyyy:HH:mm:ss")),
                F.try_to_timestamp(F.col(c), F.lit("yyyy-MM-dd HH:mm:ss")),
                F.try_to_timestamp(F.col(c), F.lit("dd/MM/yyyy HH:mm:ss")),
                F.try_to_timestamp(F.col(c), F.lit("yyyyMMddHHmmss"))
            )
        )

    return df

# Reexecutar com segurança
df_00 = padronizar_df_atraso(df_atrasos_alvo)

# print("✅ Padronização concluída com segurança.")
# print("✅ Total de colunas:", len(df_00.columns))
# df_00.printSchema()


In [25]:
#transformação da variável DAT_REFERENCIA_ATRASO em safra de atraso

df_00 = df_00.withColumn("SAFRA_ATRASO", F.date_format(F.col("DAT_REFERENCIA_ATRASO"), "yyyyMM"))

In [26]:
# df_00.groupBy("SAFRA_ATRASO").count().orderBy("SAFRA_ATRASO").show(50, truncate=False)

In [27]:
df_00 = df_00.filter(~F.col("SAFRA_ATRASO").isin(["202502", "202503"]))
# print(f"Linhas totais: {df_00.count():,}")
# print(f"CPFs únicos: {df_00.select('NUM_CPF').distinct().count():,}")

##Verificação dos nulos

In [28]:
# from pyspark.sql import functions as F

# df = df_00

# total_registros = df.count()

# diagnostico = (
#     df
#     .select([
#         F.lit(c).alias("coluna"),
#         F.count(F.lit(1)).alias("qtd_registros"),
#         F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias("qtd_nulos"),
#         F.countDistinct(F.col(c)).alias("qtd_distintos")
#     ])
#     for c in df.columns
# )

# # União de todas as colunas
# df_diagnostico = None
# for d in diagnostico:
#     if df_diagnostico is None:
#         df_diagnostico = d
#     else:
#         df_diagnostico = df_diagnostico.unionByName(d)

# # Métricas percentuais
# df_diagnostico = (
#     df_diagnostico
#     .withColumn(
#         "pct_nulos",
#         F.round(F.col("qtd_nulos") / F.col("qtd_registros") * 100, 2)
#     )
#     .withColumn(
#         "pct_distintos",
#         F.round(F.col("qtd_distintos") / F.col("qtd_registros") * 100, 2)
#     )
#     .orderBy(F.desc("pct_nulos"))
# )

# df_diagnostico.show(truncate=False)

In [29]:
cols_nao_comportamentais = [
    # Identificadores técnicos
    "NUM_FATURA_HASH_ATRASO",
    "NUM_ENT_SEQ_FATURA_ATRASO",
    "NUM_BILL_SEQ_FAT_ATRASO",
    "NUM_SEQ_ACORDO_FAT_ATRASO",
    "DW_NUM_CLIENTE_ATRASO",

    # mais 99% de nulos
    "DAT_CANCELAMENTO_FAT_ATRASO",
    "DAT_ALTERACAO_VCTO_FAT_ATRASO"
]
df_00 = df_00.drop(*cols_nao_comportamentais)

In [30]:
# df_00.printSchema()
df_00.show(5)

+-----------+---------------------+---------------+--------------------+-------------------------------+--------------+---------------+----------------------------+----------------+----------------------------+----------------------------+--------------------------+-------------------------------+--------------------------+---------------------+---------------------------------+-----------------------------------+----------------------------+----------------------+-------------------------+---------------------+-----------------------------+--------------------------+-------------+--------------+---------------+--------------+-----------------------+-----------------+----------------------+--------------------+----------------------+---------------------+-----------------------+------------------------------+---------------------+-------------------------+----------------------+-----------------------------+----------------------------+------------------------+-------------------------

In [31]:
# Filtrar e ordenar df_01
# df_filtered_ordered = df_01.filter(F.col("NUM_CPF") == "ZXY9X7ZYXNU") \
#                          .orderBy(F.col("DAT_STATUS_PAGAMENTO_PAG").asc())

# # Exibir os resultados
# df_filtered_ordered.show(truncate=False)

In [32]:
len(df_00.columns)

44

##Início criação variáveis

Imports e criação da SAFRA

1️⃣ Setup e SAFRA

In [33]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df = (
    df_00
    .withColumn("SAFRA_ATRASO", F.date_format("DAT_REFERENCIA_ATRASO", "yyyyMM"))
)


2️⃣ Variáveis base de atraso (linha → insumo)

In [34]:
df = (
    df
    .withColumn(
        "DIAS_ATRASO",
        F.datediff("DAT_REFERENCIA_ATRASO", "DAT_VENCIMENTO_FAT_ATRASO")
    )
    .withColumn(
        "DIAS_BASE_CLIENTE",
        F.datediff("DAT_REFERENCIA_ATRASO", "DAT_ATIVACAO_CONTA_CLI_ATRASO")
    )
)


3️⃣ INTENSIDADE DO ATRASO (por safra)

In [35]:
df_intensidade = (
    df
    .groupBy("NUM_CPF", "SAFRA_ATRASO")
    .agg(
        F.count("*").alias("QTD_FATURAS_ATRASO"),
        F.max("DIAS_ATRASO").alias("MAX_DIAS_ATRASO"),
        F.avg("DIAS_ATRASO").alias("AVG_DIAS_ATRASO"),
        F.min("DIAS_ATRASO").alias("MIN_DIAS_ATRASO"),
        F.stddev("DIAS_ATRASO").alias("STD_DIAS_ATRASO"),

        F.expr("percentile(DIAS_ATRASO, 0.5)").alias("MEDIANA_DIAS_ATRASO"),
        F.expr("percentile(DIAS_ATRASO, 0.9)").alias("P90_DIAS_ATRASO")
    )
)


👉 Aqui você já diferencia:

atraso pontual vs crônico

atraso estável vs volátil

4️⃣ SEVERIDADE CATEGÓRICA DO ATRASO

In [36]:
df_intensidade = (
    df_intensidade
    .withColumn(
        "SEVERIDADE_ATRASO",
        F.when(F.col("MAX_DIAS_ATRASO") <= 5, "BAIXA")
         .when(F.col("MAX_DIAS_ATRASO") <= 30, "MEDIA")
         .when(F.col("MAX_DIAS_ATRASO") <= 60, "ALTA")
         .otherwise("CRITICA")
    )
)


5️⃣ FINANCEIRAS – VALOR, PESO E PRESSÃO

In [37]:
df_valores = (
    df
    .groupBy("NUM_CPF", "SAFRA_ATRASO")
    .agg(
        F.sum("VAL_FAT_LIQUIDO_ATRASO").alias("SUM_FAT_LIQ"),
        F.sum("VAL_FAT_BRUTO_ATRASO").alias("SUM_FAT_BRUTO"),
        F.sum("VAL_FAT_ABERTO_ATRASO").alias("SUM_ABERTO"),
        F.sum("VAL_FAT_ABERTO_LIQ_ATRASO").alias("SUM_ABERTO_LIQ"),
        F.sum("VAL_FAT_PAGAMENTO_BRUTO_ATRASO").alias("SUM_PAGO_BRUTO"),
        F.sum("VAL_MULTA_JUROS_ATRASO").alias("SUM_JUROS"),
        F.sum("VAL_MULTA_CANCELAMENTO_ATRASO").alias("SUM_MULTA_CANCEL"),
        F.sum("VAL_PARC_APARELHO_LIQ_ATRASO").alias("SUM_PARC_APARELHO")
    )
)


Ratios financeiros (muito poder preditivo)

In [95]:
df_valores = (
    df_valores
    .withColumn("RATIO_ABERTO_LIQ", F.try_divide(F.col("SUM_ABERTO_LIQ"), F.col("SUM_FAT_LIQ")))
    .withColumn("RATIO_JUROS", F.try_divide(F.col("SUM_JUROS"), F.col("SUM_FAT_LIQ")))
    .withColumn("RATIO_PAGAMENTO", F.try_divide(F.col("SUM_PAGO_BRUTO"), F.col("SUM_FAT_BRUTO")))
)

6️⃣ PERSISTÊNCIA E RECORRÊNCIA (por CPF)

In [68]:
window_cpf = Window.partitionBy("NUM_CPF")


In [69]:
df_persistencia = (
    df
    .groupBy("NUM_CPF", "SAFRA_ATRASO")
    .agg(
        F.max("DIAS_ATRASO").alias("MAX_DIAS_ATRASO_SAFRA")
    )
    .withColumn(
        "FLG_ATRASO_SAFRA",
        F.when(F.col("MAX_DIAS_ATRASO_SAFRA") > 0, 1).otherwise(0)
    )
    .withColumn(
        "QTD_SAFRAS_ATRASO",
        F.sum("FLG_ATRASO_SAFRA").over(window_cpf)
    )
)


7️⃣ FLAGS OPERACIONAIS (agregação lógica)

In [70]:
# df.groupBy("IND_WO_ATRASO").agg(F.countDistinct("NUM_CPF").alias("count_cpf")).show()

In [71]:
# df.groupBy("IND_PDD_ATRASO").agg(F.countDistinct("NUM_CPF").alias("count_cpf")).show()

In [72]:
# df.groupBy("IND_ACA_ATRASO").agg(F.countDistinct("NUM_CPF").alias("count_cpf")).show()

In [73]:
# df.groupBy("IND_FRAUDE_ATRASO").agg(F.countDistinct("NUM_CPF").alias("count_cpf")).show()

In [74]:
# df.groupBy("IND_PRIMEIRA_FAT_ATRASO").agg(F.countDistinct("NUM_CPF").alias("count_cpf")).show()

In [75]:
df_flags = (
    df
    .groupBy("NUM_CPF", "SAFRA_ATRASO")
    .agg(
        F.max(F.when(F.col("IND_WO_ATRASO") == "W", F.lit(1)).otherwise(F.lit(0))).alias("FLG_WO"),
        F.max(F.when(F.col("IND_PDD_ATRASO") == "S", F.lit(1)).otherwise(F.lit(0))).alias("FLG_PDD"),
        F.max(F.when(F.col("IND_ACA_ATRASO").isin("S", "A"), F.lit(1)).otherwise(F.lit(0))).alias("FLG_ACAO_COB"),
        F.max(F.when(F.col("IND_FRAUDE_ATRASO") == "S", F.lit(1)).otherwise(F.lit(0))).alias("FLG_FRAUDE"),
        F.max(F.when(F.col("IND_PRIMEIRA_FAT_ATRASO") == "S", F.lit(1)).otherwise(F.lit(0))).alias("FLG_PRIMEIRA_FAT")
    )
    .fillna(0, subset=["FLG_WO", "FLG_PDD", "FLG_ACAO_COB", "FLG_FRAUDE", "FLG_PRIMEIRA_FAT"])
)

In [76]:
w = Window.partitionBy("NUM_CPF", "SAFRA_ATRASO").orderBy(F.desc("DAT_REFERENCIA_ATRASO"))

df_cat = (
    df
    .withColumn("RN", F.row_number().over(w))
    .filter("RN = 1")
    .select(
        "NUM_CPF", "SAFRA_ATRASO",
        "DW_UN_NEGOCIO_ATRASO",
        "DW_AREA_ATRASO",
        "DW_TIPO_CLIENTE_CONTA_ATRASO",
        "DW_CICLO_ATRASO",
        "DW_TIPO_FATURAMENTO_ATRASO",
        "DW_OFERTA_ATRASO",
        "DW_FAIXA_AGING_FATURA_ATRASO",
        "DW_FAIXA_AGING_DIVIDA_ATRASO"
    )
)


9️⃣ BOOK FINAL (≈ 70–90 variáveis)

#Book de atrasos

In [96]:
df_book_atrasos = (
    df_intensidade
    .join(df_valores, ["NUM_CPF", "SAFRA_ATRASO"], "left")
    .join(df_persistencia, ["NUM_CPF", "SAFRA_ATRASO"], "left")
    .join(df_flags, ["NUM_CPF", "SAFRA_ATRASO"], "left")
    .join(df_cat, ["NUM_CPF", "SAFRA_ATRASO"], "left")
)

In [78]:
# df_book_atrasos.printSchema()

In [79]:
# len(df_book_atrasos.columns)

🔎 Garantia de granularidade

In [80]:
# df_book_atrasos.groupBy("NUM_CPF", "SAFRA_ATRASO").count().filter("count > 1").show()


#Construção das janelas temporárias de 1, 3. 6, 9, 12 meses

## Função genérica para janelas temporais VERSÃO COMPLETA

In [81]:
def window_features(df, months):

    w = Window.partitionBy("NUM_CPF") \
              .orderBy("SAFRA_ATRASO") \
              .rowsBetween(-months + 1, 0)

    df_w = df.select (
        "NUM_CPF",
        "SAFRA_ATRASO",

        # =========================
        # INTENSIDADE/MÉDIA (nível médio)
        # =========================
        F.avg("QTD_FATURAS_ATRASO").over(w).alias(f"mean_QTD_FATURAS_ATRASO_u{months}m"),
        F.avg("SUM_ABERTO").over(w).alias(f"mean_SUM_ABERTO_u{months}m"),
        F.avg("QTD_SAFRAS_ATRASO").over(w).alias(f"mean_QTD_SAFRAS_ATRASO_u{months}m"),


        #F.avg("MAX_DIAS_ATRASO").over(w).alias(f"mean_MAX_DIAS_ATRASO_u{months}m"),
        #F.avg("AVG_DIAS_ATRASO").over(w).alias(f"mean_AVG_DIAS_ATRASO_u{months}m"),
        #F.avg("MIN_DIAS_ATRASO").over(w).alias(f"mean_MIN_DIAS_ATRASO_u{months}m"),
        #F.avg("STD_DIAS_ATRASO").over(w).alias(f"mean_STD_DIAS_ATRASO_u{months}m"),
        #F.avg("MEDIANA_DIAS_ATRASO").over(w).alias(f"mean_MEDIANA_DIAS_ATRASO_u{months}m"),
        #F.avg("P90_DIAS_ATRASO").over(w).alias(f"mean_P90_DIAS_ATRASO_u{months}m"),
        #F.avg("SUM_FAT_LIQ").over(w).alias(f"mean_SUM_FAT_LIQ_u{months}m"),
        #F.avg("SUM_FAT_BRUTO").over(w).alias(f"mean_SUM_FAT_BRUTO_u{months}m"),
        #F.avg("SUM_ABERTO_LIQ").over(w).alias(f"mean_SUM_ABERTO_LIQ_u{months}m"),
        #F.avg("SUM_PAGO_BRUTO").over(w).alias(f"mean_SUM_PAGO_BRUTO_u{months}m"),
        #F.avg("SUM_JUROS").over(w).alias(f"mean_SUM_JUROS_u{months}m"),
        #F.avg("SUM_MULTA_CANCEL").over(w).alias(f"mean_SUM_MULTA_CANCEL_u{months}m"),
        #F.avg("SUM_PARC_APARELHO").over(w).alias(f"mean_SUM_PARC_APARELHO_u{months}m"),
        #F.avg("RATIO_ABERTO_LIQ").over(w).alias(f"mean_RATIO_ABERTO_LIQ_u{months}m"),
        #F.avg("RATIO_JUROS").over(w).alias(f"mean_RATIO_JUROS_u{months}m"),
        #F.avg("RATIO_PAGAMENTO").over(w).alias(f"mean_RATIO_PAGAMENTO_u{months}m"),
        #F.avg("MAX_DIAS_ATRASO_SAFRA").over(w).alias(f"mean_MAX_DIAS_ATRASO_SAFRA_u{months}m"),



        # =========================
        #  FREQUENCIA/CONTAGEM
        # =========================
        F.count("QTD_FATURAS_ATRASO").over(w).alias(f"freq_QTD_FATURAS_ATRASO_u{months}m"),
        #F.count("MAX_DIAS_ATRASO").over(w).alias(f"freq_MAX_DIAS_ATRASO_u{months}m"),
        #F.count("MIN_DIAS_ATRASO").over(w).alias(f"freq_MIN_DIAS_ATRASO_u{months}m"),
        #F.count("MAX_DIAS_ATRASO_SAFRA").over(w).alias(f"freq_MAX_DIAS_ATRASO_SAFRA_u{months}m"),
        F.count("QTD_SAFRAS_ATRASO").over(w).alias(f"freq_QTD_SAFRAS_ATRASO_u{months}m"),

        # =========================
        # VOLATILIDADE/VARIABILIDADE
        # =========================

        # AVG_DIAS_ATRASO
        #F.stddev("AVG_DIAS_ATRASO").over(w).alias(f"std_AVG_DIAS_ATRASO_u{months}m"),
        # STD_DIAS_ATRASO
        #F.stddev("STD_DIAS_ATRASO").over(w).alias(f"std_STD_DIAS_ATRASO_u{months}m"),
        # MEDIANA_DIAS_ATRASO
        #F.stddev("MEDIANA_DIAS_ATRASO").over(w).alias(f"std_MEDIANA_DIAS_ATRASO_u{months}m"),
        # P90_DIAS_ATRASO
        #F.stddev("P90_DIAS_ATRASO").over(w).alias(f"std_P90_DIAS_ATRASO_u{months}m"),
        # SUM_FAT_LIQ
        #F.stddev("SUM_FAT_LIQ").over(w).alias(f"std_SUM_FAT_LIQ_u{months}m"),
        # SUM_FAT_BRUTO
        #F.stddev("SUM_FAT_BRUTO").over(w).alias(f"std_SUM_FAT_BRUTO_u{months}m"),
        # SUM_ABERTO
        F.stddev("SUM_ABERTO").over(w).alias(f"std_SUM_ABERTO_u{months}m"),
        # SUM_ABERTO_LIQ
        #F.stddev("SUM_ABERTO_LIQ").over(w).alias(f"std_SUM_ABERTO_LIQ_u{months}m"),
        # SUM_PAGO_BRUTO
        #F.stddev("SUM_PAGO_BRUTO").over(w).alias(f"std_SUM_PAGO_BRUTO_u{months}m"),
        # SUM_JUROS
        #F.stddev("SUM_JUROS").over(w).alias(f"std_SUM_JUROS_u{months}m"),
        # SUM_MULTA_CANCEL
        #F.stddev("SUM_MULTA_CANCEL").over(w).alias(f"std_SUM_MULTA_CANCEL_u{months}m"),
        # SUM_PARC_APARELHO
        #F.stddev("SUM_PARC_APARELHO").over(w).alias(f"std_SUM_PARC_APARELHO_u{months}m"),
        # RATIO_ABERTO_LIQ
        #F.stddev("RATIO_ABERTO_LIQ").over(w).alias(f"std_RATIO_ABERTO_LIQ_u{months}m"),
        # RATIO_JUROS
        #F.stddev("RATIO_JUROS").over(w).alias(f"std_RATIO_JUROS_u{months}m"),
        # RATIO_PAGAMENTO
        #F.stddev("RATIO_PAGAMENTO").over(w).alias(f"std_RATIO_PAGAMENTO_u{months}m"),


        # =========================
        # VOLUNE/SOMA
        # =========================
        # QTD_FATURAS_ATRASO
        F.sum("QTD_FATURAS_ATRASO").over(w).alias(f"cumsum_QTD_FATURAS_ATRASO_u{months}m"),
        # MAX_DIAS_ATRASO
        #F.sum("MAX_DIAS_ATRASO").over(w).alias(f"cumsum_MAX_DIAS_ATRASO_u{months}m"),
        # AVG_DIAS_ATRASO
        #F.sum("AVG_DIAS_ATRASO").over(w).alias(f"cumsum_AVG_DIAS_ATRASO_u{months}m"),
        # MIN_DIAS_ATRASO
        #F.sum("MIN_DIAS_ATRASO").over(w).alias(f"cumsum_MIN_DIAS_ATRASO_u{months}m"),
        # STD_DIAS_ATRASO
        #F.sum("STD_DIAS_ATRASO").over(w).alias(f"cumsum_STD_DIAS_ATRASO_u{months}m"),
        # MEDIANA_DIAS_ATRASO
        #F.sum("MEDIANA_DIAS_ATRASO").over(w).alias(f"cumsum_MEDIANA_DIAS_ATRASO_u{months}m"),
        # P90_DIAS_ATRASO
        #F.sum("P90_DIAS_ATRASO").over(w).alias(f"cumsum_P99_DIAS_ATRASO_u{months}m"),
        # SUM_FAT_LIQ
        #F.sum("SUM_FAT_LIQ").over(w).alias(f"cumsum_SUM_FAT_LIQ_u{months}m"),
        # SUM_FAT_BRUTO
        #F.sum("SUM_FAT_BRUTO").over(w).alias(f"cumsum_SUM_FAT_BRUTO_u{months}m"),
        # SUM_ABERTO
        F.sum("SUM_ABERTO").over(w).alias(f"cumsum_SUM_ABERTO_u{months}m"),
        # SUM_ABERTO_LIQ
        #F.sum("SUM_ABERTO_LIQ").over(w).alias(f"cumsum_SUM_ABERTO_LIQ_u{months}m"),
        # SUM_PAGO_BRUTO
        #F.sum("SUM_PAGO_BRUTO").over(w).alias(f"cumsum_SUM_PAGO_BRUTO_u{months}m"),
        # SUM_JUROS
        #F.sum("SUM_JUROS").over(w).alias(f"cumsum_SUM_JUROS_u{months}m"),
        # SUM_MULTA_CANCEL
        #F.sum("SUM_MULTA_CANCEL").over(w).alias(f"cumsum_SUM_MULTA_CANCEL_u{months}m"),
        # SUM_PARC_APARELHO
        #F.sum("SUM_PARC_APARELHO").over(w).alias(f"cumsum_SUM_PARC_APARELHO_u{months}m"),
        # RATIO_ABERTO_LIQ
        #F.sum("RATIO_ABERTO_LIQ").over(w).alias(f"cumsum_RATIO_ABERTO_LIQ_u{months}m"),
        # RATIO_JUROS
        #F.sum("RATIO_JUROS").over(w).alias(f"cumsum_RATIO_JUROS_u{months}m"),
        # RATIO_PAGAMENTO
        #F.sum("RATIO_PAGAMENTO").over(w).alias(f"cumsum_RATIO_PAGAMENTO_u{months}m"),
        # MAX_DIAS_ATRASO_SAFRA
        #F.sum("MAX_DIAS_ATRASO_SAFRA").over(w).alias(f"cumsum_MAX_DIAS_ATRASO_SAFRA_u{months}m"),
        # QTD_SAFRAS_ATRASO
        #F.sum("QTD_SAFRAS_ATRASO").over(w).alias(f"cumsum_QTD_SAFRAS_ATRASO_u{months}m"),




        # =========================
        # EXTREMOS/MÍNIMO MÁXIMO
        # =========================
        #F.min("QTD_FATURAS_ATRASO").over(w).alias(f"min_QTD_FATURAS_ATRASO_u{months}m"),
        #F.max("QTD_FATURAS_ATRASO").over(w).alias(f"max_QTD_FATURAS_ATRASO_u{months}m"),

        #F.min("MAX_DIAS_ATRASO").over(w).alias(f"min_MAX_DIAS_ATRASO_u{months}m"),
        #F.max("MAX_DIAS_ATRASO").over(w).alias(f"max_MAX_DIAS_ATRASO_u{months}m"),

        #F.min("AVG_DIAS_ATRASO").over(w).alias(f"min_AVG_DIAS_ATRASO_u{months}m"),
        #F.max("AVG_DIAS_ATRASO").over(w).alias(f"max_AVG_DIAS_ATRASO_u{months}m"),

        #F.min("MIN_DIAS_ATRASO").over(w).alias(f"min_MIN_DIAS_ATRASO_u{months}m"),
        #F.max("MIN_DIAS_ATRASO").over(w).alias(f"max_MIN_DIAS_ATRASO_u{months}m"),

        #F.min("STD_DIAS_ATRASO").over(w).alias(f"min_STD_DIAS_ATRASO_u{months}m"),
        #F.max("STD_DIAS_ATRASO").over(w).alias(f"max_STD_DIAS_ATRASO_u{months}m"),

        #F.min("MEDIANA_DIAS_ATRASO").over(w).alias(f"min_MEDIANA_DIAS_ATRASO_u{months}m"),
        #F.max("MEDIANA_DIAS_ATRASO").over(w).alias(f"max_MEDIANA_DIAS_ATRASO_u{months}m"),

        #F.min("P90_DIAS_ATRASO_u").over(w).alias(f"min_P90_DIAS_ATRASO_u{months}m"),
        #F.max("P90_DIAS_ATRASO").over(w).alias(f"max_P90_DIAS_ATRASO_u{months}m"),

        #F.min("SUM_FAT_LIQ").over(w).alias(f"min_SUM_FAT_LIQ_u{months}m"),
        #F.max("SUM_FAT_LIQ").over(w).alias(f"max_SUM_FAT_LIQ_u{months}m"),

        #F.min("SUM_FAT_BRUTO").over(w).alias(f"min_SUM_FAT_BRUTO_u{months}m"),
        #F.max("SUM_FAT_BRUTO").over(w).alias(f"max_SUM_FAT_BRUTO_u{months}m"),

        #F.min("SUM_ABERTO").over(w).alias(f"min_SUM_ABERTO_u{months}m"),
        #F.max("SUM_ABERTO").over(w).alias(f"max_SUM_ABERTO_u{months}m"),

        #F.min("SUM_ABERTO_LIQ").over(w).alias(f"min_SUM_ABERTO_LIQ_u{months}m"),
        #F.max("SUM_ABERTO_LIQ").over(w).alias(f"max_SUM_ABERTO_LIQ_u{months}m"),

        #F.min("SUM_PAGO_BRUTO").over(w).alias(f"min_SUM_PAGO_BRUTO_u{months}m"),
        #F.max("SUM_PAGO_BRUTO").over(w).alias(f"max_SUM_PAGO_BRUTO_u{months}m"),

        #F.min("SUM_JUROS").over(w).alias(f"min_SUM_JUROS_u{months}m"),
        #F.max("SUM_JUROS").over(w).alias(f"max_SUM_JUROS_u{months}m"),

        #F.min("SUM_MULTA_CANCEL").over(w).alias(f"min_SUM_MULTA_CANCEL_u{months}m"),
        #F.max("SUM_MULTA_CANCEL").over(w).alias(f"max_SUM_MULTA_CANCEL_u{months}m"),

        #F.min("SUM_PARC_APARELHO").over(w).alias(f"min_SUM_PARC_APARELHO_u{months}m"),
        #F.max("SUM_PARC_APARELHO").over(w).alias(f"max_SUM_PARC_APARELHO_u{months}m"),

        #F.min("RATIO_ABERTO_LIQ").over(w).alias(f"min_RATIO_ABERTO_LIQ_u{months}m"),
        #F.max("RATIO_ABERTO_LIQ").over(w).alias(f"max_RATIO_ABERTO_LIQ_u{months}m"),

        #F.min("RATIO_JUROS").over(w).alias(f"min_RATIO_JUROS_u{months}m"),
        #F.max("RATIO_JUROS").over(w).alias(f"max_RATIO_JUROS_u{months}m"),

        #F.min("RATIO_PAGAMENTO").over(w).alias(f"min_RATIO_PAGAMENTO_u{months}m"),
        #F.max("RATIO_PAGAMENTO").over(w).alias(f"max_RATIO_PAGAMENTO_u{months}m"),

        #F.min("MAX_DIAS_ATRASO_SAFRA").over(w).alias(f"min_MAX_DIAS_ATRASO_SAFRA_u{months}m"),
        #F.max("MAX_DIAS_ATRASO_SAFRA").over(w).alias(f"max_MAX_DIAS_ATRASO_SAFRA_u{months}m"),

        #F.min("QTD_SAFRAS_ATRASO").over(w).alias(f"min_QTD_SAFRAS_ATRASO_u{months}m"),
        #F.max("QTD_SAFRAS_ATRASO").over(w).alias(f"max_QTD_SAFRAS_ATRASO_u{months}m"),


        # =========================
        # DISTRIBUIÇÃO (cauda)
        # =========================
        F.skewness("QTD_FATURAS_ATRASO").over(w).alias(f"skewness_QTD_FATURAS_ATRASO_u{months}m"),
        F.skewness("SUM_ABERTO").over(w).alias(f"skewness_SUM_ABERTO_u{months}m"),
        F.skewness("QTD_SAFRAS_ATRASO").over(w).alias(f"skewness_QTD_SAFRAS_ATRASO_u{months}m"),






            # =========================
            # QUARTIZ
            # =========================


            # =========================
            # PERCENTIL
            # =========================

            #F.percentile_approx("PCT_GRUPO_CARTAO_DOMINANTE", 0.25).over(w).alias(f"p25_PCT_GRUPO_CARTAO_DOMINANTE_u{months}m"),
            #F.percentile_approx("PCT_GRUPO_CARTAO_DOMINANTE", 0.50).over(w).alias(f"p50_PCT_GRUPO_CARTAO_DOMINANTE_u{months}m"),
            #F.percentile_approx("PCT_GRUPO_CARTAO_DOMINANTE", 0.75).over(w).alias(f"p75_PCT_GRUPO_CARTAO_DOMINANTE_u{months}m"),
            #F.percentile_approx("PCT_GRUPO_CARTAO_DOMINANTE", 0.90).over(w).alias(f"p90_PCT_GRUPO_CARTAO_DOMINANTE_u{months}m"),
            #F.percentile_approx("PCT_GRUPO_CARTAO_DOMINANTE", 0.95).over(w).alias(f"p95_PCT_GRUPO_CARTAO_DOMINANTE_u{months}m"),
            #F.percentile_approx("PCT_GRUPO_CARTAO_DOMINANTE", 0.99).over(w).alias(f"p99_PCT_GRUPO_CARTAO_DOMINANTE_u{months}m"),

        # =========================
        # PERSISTÊNCIA
        # =========================
        F.sum("FLG_ATRASO_SAFRA").over(w).alias(f"qtd_meses_FLG_ATRASO_SAFRA_u{months}m"),




        # =========================
        # FLAGS TEMPORAIS
        # =========================
        F.max("FLG_ATRASO_SAFRA").over(w).alias(f"FLG_ATRASO_SAFRA_u{months}m"),


        # =========================
        # RANKING
        # =========================

        # =========================
        # FEATURES CRUZADAS
        # =========================

        # =========================
        # VARIAVEIS DE COMPARAÇÃO
        # Comparamos janelas curtas com janelas longas para identificar mudanças de comportamento.
        # =========================

        # =========================
        #       DETERIORAÇÃO/MELHORA
        # =========================


        # =========================
        #       ACELERAÇÃO
        #       Exemplo: Se a razão for > 1, o cliente está gastando mais do que o seu padrão histórico.
        # =========================


        # =========================
        #       DESACELERAÇÃO
        #       Exemplo: Se a razão for < 1, o cliente está gastando menos do que o seu padrão histórico.
        # =========================



    )

    return df_w


## Construção de janela de 1 mês do datraframe df_book

In [97]:
df_m1 = window_features(df_book_atrasos, 1)

In [83]:
df_m1.printSchema()

root
 |-- NUM_CPF: string (nullable = true)
 |-- SAFRA_ATRASO: string (nullable = true)
 |-- mean_QTD_FATURAS_ATRASO_u1m: double (nullable = true)
 |-- mean_SUM_ABERTO_u1m: double (nullable = true)
 |-- mean_QTD_SAFRAS_ATRASO_u1m: double (nullable = true)
 |-- freq_QTD_FATURAS_ATRASO_u1m: long (nullable = false)
 |-- freq_QTD_SAFRAS_ATRASO_u1m: long (nullable = false)
 |-- std_SUM_ABERTO_u1m: double (nullable = true)
 |-- cumsum_QTD_FATURAS_ATRASO_u1m: long (nullable = true)
 |-- cumsum_SUM_ABERTO_u1m: double (nullable = true)
 |-- skewness_QTD_FATURAS_ATRASO_u1m: double (nullable = true)
 |-- skewness_SUM_ABERTO_u1m: double (nullable = true)
 |-- skewness_QTD_SAFRAS_ATRASO_u1m: double (nullable = true)
 |-- qtd_meses_FLG_ATRASO_SAFRA_u1m: long (nullable = true)
 |-- FLG_ATRASO_SAFRA_u1m: integer (nullable = true)



In [84]:
# numero de colunas do dataframe pyspark
len(df_m1.columns)


15

## Construção de janela de 3 meses do datraframe df_book

In [98]:
df_m3 = window_features(df_book_atrasos, 3)

## Construção de janela de 6 meses do datraframe df_book

In [99]:
df_m6 = window_features(df_book_atrasos, 6)

## Construção de janela de 9 meses do datraframe df_book

---



In [100]:
df_m9 = window_features(df_book_atrasos, 9)

## Construção de janela de 12 meses do datraframe df_book



---



In [101]:
df_m12 = window_features(df_book_atrasos, 12)

## Book final com as janelas

In [102]:
df_features = df_book_atrasos \
    .select("*") \
    .join(df_m1, ["NUM_CPF", "SAFRA_ATRASO"], "left") \
    .join(df_m3, ["NUM_CPF", "SAFRA_ATRASO"], "left") \
    .join(df_m6, ["NUM_CPF", "SAFRA_ATRASO"], "left") \
    .join(df_m9, ["NUM_CPF", "SAFRA_ATRASO"], "left") \
    .join(df_m12, ["NUM_CPF", "SAFRA_ATRASO"], "left")

In [90]:
df_features.printSchema()

root
 |-- NUM_CPF: string (nullable = true)
 |-- SAFRA_ATRASO: string (nullable = true)
 |-- QTD_FATURAS_ATRASO: long (nullable = false)
 |-- MAX_DIAS_ATRASO: integer (nullable = true)
 |-- AVG_DIAS_ATRASO: double (nullable = true)
 |-- MIN_DIAS_ATRASO: integer (nullable = true)
 |-- STD_DIAS_ATRASO: double (nullable = true)
 |-- MEDIANA_DIAS_ATRASO: double (nullable = true)
 |-- P90_DIAS_ATRASO: double (nullable = true)
 |-- SEVERIDADE_ATRASO: string (nullable = false)
 |-- SUM_FAT_LIQ: double (nullable = true)
 |-- SUM_FAT_BRUTO: double (nullable = true)
 |-- SUM_ABERTO: double (nullable = true)
 |-- SUM_ABERTO_LIQ: double (nullable = true)
 |-- SUM_PAGO_BRUTO: double (nullable = true)
 |-- SUM_JUROS: double (nullable = true)
 |-- SUM_MULTA_CANCEL: double (nullable = true)
 |-- SUM_PARC_APARELHO: double (nullable = true)
 |-- RATIO_ABERTO_LIQ: double (nullable = true)
 |-- RATIO_JUROS: double (nullable = true)
 |-- RATIO_PAGAMENTO: double (nullable = true)
 |-- MAX_DIAS_ATRASO_SAFRA:

In [91]:
len (df_features.columns)

102

In [103]:
df_book_atraso_202501 = df_features.filter(F.col('SAFRA_ATRASO') == 202501)
df_book_atraso_202412 = df_features.filter(F.col('SAFRA_ATRASO') == 202412)
df_book_atraso_202411 = df_features.filter(F.col('SAFRA_ATRASO') == 202411)
df_book_atraso_202410 = df_features.filter(F.col('SAFRA_ATRASO') == 202410)

In [93]:
# spark.conf.set("spark.sql.parquet.compression.codec", "snappy")
# spark.conf.set("spark.sql.parquet.mergeSchema", "false")
# spark.conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
# spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")

In [105]:
from google.colab import drive
drive.mount("/content/drive")
from pyspark.sql import functions as F
from pyspark.sql.types import NullType
import math

def save_parquet_wide_df(
    df,
    base_path,
    block_size=300,
    n_partitions=24,
    mode="overwrite"
):
    """
    Salva DataFrame Spark muito largo (ex: 3000+ colunas)
    em blocos de colunas no Google Drive (ou FS estável).

    Cada bloco é um parquet separado.
    """

    # =========================
    # 1. Sanear schema
    # =========================
    for field in df.schema.fields:
        if isinstance(field.dataType, NullType):
            df = df.withColumn(field.name, F.lit(None).cast("string"))

    # =========================
    # 2. Repartition (NUNCA coalesce)
    # =========================
    df = df.repartition(n_partitions)

    # =========================
    # 3. Preparar blocos
    # =========================
    cols = df.columns
    total_cols = len(cols)
    n_blocks = math.ceil(total_cols / block_size)

    print(f"→ Total colunas: {total_cols}")
    print(f"→ Blocos: {n_blocks} (≈ {block_size} colunas/bloco)")
    print(f"→ Partições: {n_partitions}")

    # =========================
    # 4. Salvar metadados
    # =========================
    spark.createDataFrame(
        [(i, c) for i, c in enumerate(cols)],
        ["ordem", "coluna"]
    ).write.mode(mode).parquet(f"{base_path}/metadata")

    # =========================
    # 5. Salvar blocos
    # =========================
    for i in range(n_blocks):
        start = i * block_size
        end = min(start + block_size, total_cols)
        block_cols = cols[start:end]

        block_path = f"{base_path}/block{i:03d}"

        print(f"→ Salvando bloco {i+1}/{n_blocks} | cols {start}:{end}")

        (
            df.select(*block_cols)
              .write
              .mode(mode)
              .parquet(block_path)
        )

    print("✅ Salvamento concluído com sucesso.")



save_parquet_wide_df(
    df=df_book_atraso_202501,
    #base_path="/content/drive/MyDrive/HACKATHON_2025/recargas_book_202501",
    base_path="/content/drive/MyDrive/HACKATHON_2025/atrasos_book_202501",
    block_size=300,     # ajuste se necessário (300–500)
    n_partitions=24     # ajuste c
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
→ Total colunas: 102
→ Blocos: 1 (≈ 300 colunas/bloco)
→ Partições: 24
→ Salvando bloco 1/1 | cols 0:102
✅ Salvamento concluído com sucesso.
